### 라이브러리 불러오기

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings(action='ignore')

In [3]:
import time
from glob import glob
from tqdm import tqdm
from scipy import interpolate

In [4]:
# conda install tensorflow

In [5]:
# conda install python=3.7

In [6]:
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Activation, Dense, LSTM, GRU, AveragePooling1D, GlobalAveragePooling1D, Dropout, Flatten, BatchNormalization
from keras.layers.convolutional import Conv1D, MaxPooling1D, AveragePooling1D 
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras import backend as K

### 전체 수위 데이터셋 구축

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [76]:
# 데이터 불러오기 
전체df = pd.read_csv('/content/drive/MyDrive/신입기수 프로젝트/Dataset/rawdata.csv')
# for colab
# /content/drive/MyDrive/신입기수 프로젝트/Dataset/rawdata.csv
# for jupyter
# /Volumes/GoogleDrive/.shortcut-targets-by-id/1rUx7_7LmBqAQTbWEWVTGvw0_d0v6N2ZT/신입기수 프로젝트/Dataset/rawdata.csv

In [77]:
수위 = 전체df['행주대교_수위']

In [78]:
수위

0         275.3
1         275.3
2         275.3
3         276.3
4         277.3
          ...  
276331      0.0
276332      0.0
276333      0.0
276334      0.0
276335      0.0
Name: 행주대교_수위, Length: 276336, dtype: float64

In [79]:
nanIndex = 수위[수위.isnull()==True].index

In [80]:
len(nanIndex)

59

In [81]:
consecutive = list(nanIndex)[:-1]
unconsecutive = list(nanIndex)[-1]
print(consecutive)
print(unconsecutive)

[140613, 140614, 140615, 140616, 140617, 140618, 140619, 140620, 140621, 140622, 140623, 140624, 140625, 140626, 140627, 140628, 140629, 140630, 140631, 140632, 140633, 140634, 140635, 140636, 140637, 140638, 140639, 140640, 140641, 140642, 140643, 140644, 140645, 140646, 140647, 140648, 140649, 140650, 140651, 140652, 140653, 140654, 140655, 140656, 140657, 140658, 140659, 140660, 140661, 140662, 140663, 140664, 140665, 140666, 140667, 140668, 140669, 140670]
236080


In [82]:
print(len(consecutive) + 1 == len(nanIndex) )

True


In [83]:
# 58개 결측치 앞 인덱스까지 인풋 생성
x = 수위.iloc[:140613]

In [84]:
x

0         275.3
1         275.3
2         275.3
3         276.3
4         277.3
          ...  
140608    374.3
140609    375.3
140610    373.3
140611    373.3
140612    375.3
Name: 행주대교_수위, Length: 140613, dtype: float64

In [85]:
# conda install scikit-learn

In [86]:
# train, test data 생성할 때 필요
from sklearn.model_selection import train_test_split
# 정규화할 때 필요
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler

### 행주대교 수위

In [114]:
start = time.time()

y_predict = []

xtrain = []
ytrain = []

w =  500 # window size
n = 58 # 예측할 데이터 개수

# xtrain 생성
# for i in range(len(x_regular)-w):
#     xtrain.append(np.array(x_regular.iloc[i:i + w, :]).astype(float))

for i in tqdm(range(len(x)-w-n)):
    xtrain.append(np.array(x.iloc[i:i + w]).astype(float))

100%|██████████| 140055/140055 [00:06<00:00, 22598.49it/s]


In [115]:
# ytrain 생성
# for i in range(w, len(x_regular)):
#     ytrain.append(np.array(x_regular.iloc[i, :]).astype(float))
ytrain = []

for i in tqdm(range(w, len(x)-n)):
  ytrain.append(np.array(x.iloc[i:i+n]).astype(float))

100%|██████████| 140055/140055 [00:05<00:00, 26241.43it/s]


In [119]:
xtrain = np.array(xtrain)
ytrain = np.array(ytrain)
print(xtrain.shape)
print(ytrain.shape)

(140055, 500)
(140055, 58)


In [120]:
df_xtrain = pd.DataFrame(xtrain, columns = [i for i in range(500)])

In [121]:
df_ytrain = pd.DataFrame(ytrain, columns = [f'R{i}' for i in range(58)])

In [122]:
df_series = pd.concat([df_xtrain, df_ytrain], axis = 1)

In [123]:
df_series.shape

(140055, 558)

In [124]:
xtrain = xtrain.reshape(140055, 500, 1)
ytrain = ytrain.reshape(140055, 58, 1)

In [125]:
# label
labels_col = [f'R{i}' for i in range(58)]
labels = df_series[labels_col]
series = df_series.drop(labels_col, axis=1)
labels.values.shape

(140055, 58)

In [126]:
# x_train, x_val, y_train, y_val 분류
x_train, x_val, y_train, y_val = train_test_split(series, labels.values, test_size = 0.2, random_state = 0)

In [127]:
x_train

,0,1,2,3,4,5,6,7,8,9,...,490,491,492,493,494,495,496,497,498,499
129790,273.3,273.3,274.3,274.3,274.3,273.3,273.3,272.3,270.3,269.3,...,278.3,278.3,277.3,277.3,276.3,276.3,275.3,282.3,312.3,333.3
65092,283.3,284.3,284.3,285.3,285.3,286.3,286.3,285.3,285.3,285.3,...,314.3,317.3,318.3,318.3,318.3,317.3,315.3,313.3,310.3,306.3
121307,262.3,262.3,263.3,264.3,265.3,266.3,266.3,266.3,266.3,266.3,...,315.3,319.3,322.3,326.3,328.3,329.3,329.3,329.3,326.3,323.3
123912,352.3,347.3,343.3,340.3,336.3,334.3,332.3,328.3,325.3,323.3,...,328.3,333.3,337.3,338.3,341.3,342.3,344.3,344.3,344.3,341.3
45845,279.3,279.3,278.3,277.3,277.3,276.3,275.3,275.3,275.3,274.3,...,316.3,316.3,315.3,315.3,314.3,314.3,314.3,313.3,313.3,313.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41993,291.3,291.3,292.3,292.3,292.3,293.3,293.3,293.3,293.3,293.3,...,294.3,293.3,292.3,292.3,291.3,291.3,290.3,290.3,289.3,288.3
97639,335.3,338.3,341.3,342.3,342.3,342.3,341.3,339.3,336.3,333.3,...,251.3,251.3,251.3,250.3,250.3,250.3,249.3,249.3,250.3,249.3
95939,263.3,263.3,263.3,263.3,262.3,262.3,261.3,260.3,259.3,258.3,...,251.3,251.3,251.3,251.3,251.3,251.3,251.3,251.3,251.3,251.3
117952,319.3,318.3,317.3,316.3,315.3,313.3,312.3,311.3,310.3,309.3,...,366.3,367.3,365.3,360.3,355.3,353.3,353.3,352.3,352.3,353.3


In [128]:
y_train

array([[348.3, 358.3, 360.3, ..., 299.3, 297.3, 296.3],
       [303.3, 300.3, 297.3, ..., 312.3, 309.3, 305.3],
       [316.3, 308.3, 306.3, ..., 274.3, 274.3, 274.3],
       ...,
       [251.3, 251.3, 250.3, ..., 248.3, 248.3, 248.3],
       [354.3, 355.3, 354.3, ..., 296.3, 296.3, 307.3],
       [297.3, 296.3, 296.3, ..., 275.3, 275.3, 275.3]])

In [129]:
# 모델링 및 모델 학습

if 'model' in locals(): 
    del model 
if 'history' in locals():
    del history


#He_normal
He_normal = keras.initializers.he_normal(seed=None)

model = Sequential()
model.add(Conv1D(filters = 8, input_shape = (w, 1), kernel_size = 2, padding='causal', dilation_rate=2, kernel_initializer = He_normal))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))  #0.2~0.5  0.8까지

model.add(Conv1D(filters = 16, kernel_size = 2, padding='causal', dilation_rate=4, kernel_initializer = He_normal))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Conv1D(filters = 32, kernel_size = 1))

model.add(Flatten())   #model.add(GlobalAveragePooling1D(pool_size=2))

model.add(Dense(50, kernel_initializer = He_normal))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(58))


def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true))) 

opt = tf.keras.optimizers.Adam(lr=0.005) #기본값 0.001
model.compile(optimizer=opt, loss=root_mean_squared_error, metrics =["accuracy"])
model.summary()



callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss',
                                            patience=5),
                keras.callbacks.ModelCheckpoint(filepath='best_model.h5',
                                                monitor='val_loss',
                                                save_best_only=True)]



history = model.fit(x_train, y_train, epochs=300, validation_data = (x_val, y_val), verbose=1, callbacks=callbacks, batch_size = 3742) #batch size..?


keras_model_best = keras.models.load_model('best_model.h5', custom_objects={'root_mean_squared_error': root_mean_squared_error})

keras_model_best.evaluate(x_val,y_val)

# y_predict.append(keras_model_best.predict(xtest))

print("time :", time.time() - start, "초")

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_6 (Conv1D)           (None, 500, 8)            24        
                                                                 
 batch_normalization_6 (Batc  (None, 500, 8)           32        
 hNormalization)                                                 
                                                                 
 activation_6 (Activation)   (None, 500, 8)            0         
                                                                 
 dropout_6 (Dropout)         (None, 500, 8)            0         
                                                                 
 conv1d_7 (Conv1D)           (None, 500, 16)           272       
                                                                 
 batch_normalization_7 (Batc  (None, 500, 16)          64        
 hNormalization)                                      

In [130]:
x_test = x[-500:]

In [131]:
x_test = np.array(x_test).reshape((1, 500, 1))

In [132]:
x_test.shape

(1, 500, 1)

In [133]:
y_predict = keras_model_best.predict(x_test)

In [134]:
# (1, 58)으로 되어 있는 차원을 (58,)으로 바꿔줘야 필요없는 대괄호가 하나 없어지면서 58개의 원소로 이루어진 리스트가 됨
# 이전에는 y_predict의 length가 1이었음(2차원 데이터이므로)
y_predict = y_predict.reshape(58,)
y_predict

array([323.29788, 325.80698, 322.70828, 323.45557, 324.58734, 323.20795,
       324.56357, 322.4321 , 324.31924, 324.0666 , 321.15707, 321.86423,
       321.92786, 322.36343, 322.48508, 322.17395, 319.9411 , 323.77213,
       322.3781 , 321.3228 , 320.67926, 321.58432, 322.29343, 320.2619 ,
       320.61612, 321.43045, 319.45724, 322.74002, 322.31033, 321.7504 ,
       321.6428 , 322.01288, 323.28357, 322.22122, 321.42163, 323.38766,
       319.7018 , 320.75198, 321.0872 , 320.32413, 322.3651 , 321.17194,
       321.4405 , 321.6619 , 322.05038, 321.3619 , 317.48856, 323.3444 ,
       321.47507, 320.17395, 322.65692, 322.1385 , 321.62134, 320.247  ,
       319.15466, 321.5745 , 322.5061 , 320.9201 ], dtype=float32)

In [135]:
수위.loc[consecutive] = y_predict

In [136]:
수위.loc[consecutive]

140613    323.297882
140614    325.806976
140615    322.708282
140616    323.455566
140617    324.587341
140618    323.207947
140619    324.563568
140620    322.432098
140621    324.319244
140622    324.066589
140623    321.157074
140624    321.864227
140625    321.927856
140626    322.363434
140627    322.485077
140628    322.173950
140629    319.941101
140630    323.772125
140631    322.378113
140632    321.322815
140633    320.679260
140634    321.584320
140635    322.293427
140636    320.261902
140637    320.616119
140638    321.430450
140639    319.457245
140640    322.740021
140641    322.310333
140642    321.750397
140643    321.642792
140644    322.012878
140645    323.283569
140646    322.221222
140647    321.421631
140648    323.387665
140649    319.701813
140650    320.751984
140651    321.087189
140652    320.324127
140653    322.365112
140654    321.171936
140655    321.440491
140656    321.661896
140657    322.050385
140658    321.361908
140659    317.488556
140660    323

In [137]:
# unconsecutive 부분 보간

unconsecutive2 = [unconsecutive-1, unconsecutive, unconsecutive+1]
수위.loc[unconsecutive2] = 수위.loc[unconsecutive2].interpolate()

In [138]:
수위.isnull().sum()

0

In [139]:
수위.to_csv('/content/drive/MyDrive/신입기수 프로젝트/Dataset/CNN_전처리/행주대교_수위_4th.csv', index = False, encoding = 'cp949')